<a href="https://colab.research.google.com/github/nathaliacastelobranco/infnet-infraestrutura-hadoop/blob/main/Transformacao_CAGED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformação para particionamento (PySpark)


In [3]:
!pip install gcsfs pyarrow fastparquet -q

In [4]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.5/gcs-connector-hadoop3-2.2.5-shaded.jar -O gcs-connector.jar

In [21]:
# ==============================================
# 1. Dependências, autenticação e configurações
# ==============================================
import os
import shutil
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, when, col, lit

from google.colab import auth
auth.authenticate_user()

anos = [2020, 2021, 2022, 2023, 2024, 2025]

In [6]:
# ========================================
# 2. Sessão Spark e Exploração dos dados
# ========================================

spark = SparkSession.builder \
    .appName("TransformacaoCAGED") \
    .config("spark.jars", "gcs-connector.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .getOrCreate()

In [11]:
df_mov_teste = spark.read.parquet("gs://bucket_caged/caged/2020/MOV/2020MOV.parquet")

In [12]:
df_mov_teste.show(5, truncate=False)

+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+-----------------+-----+----------------+--------+----+--------------+-------------------+------------------+------------------+-------------------+--------------+--------+-----------+-----------------+--------------------+---------------+----------------------+----------------------+-----------------+
|competÃªnciamov|regiÃ£o|uf |municÃ­pio|seÃ§Ã£o|subclasse|saldomovimentaÃ§Ã£o|cbo2002ocupaÃ§Ã£o|categoria|graudeinstruÃ§Ã£o|idade|horascontratuais|raÃ§acor|sexo|tipoempregador|tipoestabelecimento|tipomovimentaÃ§Ã£o|tipodedeficiÃªncia|indtrabintermitente|indtrabparcial|salÃ¡rio|tamestabjan|indicadoraprendiz|origemdainformaÃ§Ã£o|competÃªnciadec|indicadordeforadoprazo|unidadesalÃ¡riocÃ³digo|valorsalÃ¡riofixo|
+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+-----------------+-----+----------------+--------+----+--------------+----

In [15]:
df_mov_teste.printSchema()

root
 |-- competÃªnciamov: long (nullable = true)
 |-- regiÃ£o: long (nullable = true)
 |-- uf: long (nullable = true)
 |-- municÃ­pio: long (nullable = true)
 |-- seÃ§Ã£o: string (nullable = true)
 |-- subclasse: long (nullable = true)
 |-- saldomovimentaÃ§Ã£o: long (nullable = true)
 |-- cbo2002ocupaÃ§Ã£o: long (nullable = true)
 |-- categoria: long (nullable = true)
 |-- graudeinstruÃ§Ã£o: long (nullable = true)
 |-- idade: long (nullable = true)
 |-- horascontratuais: string (nullable = true)
 |-- raÃ§acor: long (nullable = true)
 |-- sexo: long (nullable = true)
 |-- tipoempregador: long (nullable = true)
 |-- tipoestabelecimento: long (nullable = true)
 |-- tipomovimentaÃ§Ã£o: long (nullable = true)
 |-- tipodedeficiÃªncia: long (nullable = true)
 |-- indtrabintermitente: long (nullable = true)
 |-- indtrabparcial: long (nullable = true)
 |-- salÃ¡rio: string (nullable = true)
 |-- tamestabjan: long (nullable = true)
 |-- indicadoraprendiz: long (nullable = true)
 |-- origemdainf

In [13]:
df_exc_teste = spark.read.parquet("gs://bucket_caged/caged/2020/EXC/2020EXC.parquet")

In [14]:
df_exc_teste.show(5, truncate=False)

+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+-----------------+-----+----------------+--------+----+--------------+-------------------+------------------+------------------+-------------------+--------------+--------+-----------+-----------------+--------------------+---------------+---------------+--------------------+----------------------+----------------------+-----------------+
|competÃªnciamov|regiÃ£o|uf |municÃ­pio|seÃ§Ã£o|subclasse|saldomovimentaÃ§Ã£o|cbo2002ocupaÃ§Ã£o|categoria|graudeinstruÃ§Ã£o|idade|horascontratuais|raÃ§acor|sexo|tipoempregador|tipoestabelecimento|tipomovimentaÃ§Ã£o|tipodedeficiÃªncia|indtrabintermitente|indtrabparcial|salÃ¡rio|tamestabjan|indicadoraprendiz|origemdainformaÃ§Ã£o|competÃªnciadec|competÃªnciaexc|indicadordeexclusÃ£o|indicadordeforadoprazo|unidadesalÃ¡riocÃ³digo|valorsalÃ¡riofixo|
+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+

In [16]:
df_exc_teste.printSchema()

root
 |-- competÃªnciamov: long (nullable = true)
 |-- regiÃ£o: long (nullable = true)
 |-- uf: long (nullable = true)
 |-- municÃ­pio: long (nullable = true)
 |-- seÃ§Ã£o: string (nullable = true)
 |-- subclasse: long (nullable = true)
 |-- saldomovimentaÃ§Ã£o: long (nullable = true)
 |-- cbo2002ocupaÃ§Ã£o: long (nullable = true)
 |-- categoria: long (nullable = true)
 |-- graudeinstruÃ§Ã£o: long (nullable = true)
 |-- idade: long (nullable = true)
 |-- horascontratuais: string (nullable = true)
 |-- raÃ§acor: long (nullable = true)
 |-- sexo: long (nullable = true)
 |-- tipoempregador: long (nullable = true)
 |-- tipoestabelecimento: long (nullable = true)
 |-- tipomovimentaÃ§Ã£o: long (nullable = true)
 |-- tipodedeficiÃªncia: long (nullable = true)
 |-- indtrabintermitente: long (nullable = true)
 |-- indtrabparcial: long (nullable = true)
 |-- salÃ¡rio: string (nullable = true)
 |-- tamestabjan: long (nullable = true)
 |-- indicadoraprendiz: long (nullable = true)
 |-- origemdainf

In [17]:
df_for_teste = spark.read.parquet("gs://bucket_caged/caged/2020/FOR/2020FOR.parquet")

In [18]:
df_for_teste.show(5, truncate=False)

+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+-----------------+-----+----------------+--------+----+--------------+-------------------+------------------+------------------+-------------------+--------------+--------+-----------+-----------------+--------------------+---------------+----------------------+----------------------+-----------------+
|competÃªnciamov|regiÃ£o|uf |municÃ­pio|seÃ§Ã£o|subclasse|saldomovimentaÃ§Ã£o|cbo2002ocupaÃ§Ã£o|categoria|graudeinstruÃ§Ã£o|idade|horascontratuais|raÃ§acor|sexo|tipoempregador|tipoestabelecimento|tipomovimentaÃ§Ã£o|tipodedeficiÃªncia|indtrabintermitente|indtrabparcial|salÃ¡rio|tamestabjan|indicadoraprendiz|origemdainformaÃ§Ã£o|competÃªnciadec|indicadordeforadoprazo|unidadesalÃ¡riocÃ³digo|valorsalÃ¡riofixo|
+---------------+-------+---+----------+-------+---------+-------------------+-----------------+---------+-----------------+-----+----------------+--------+----+--------------+----

In [19]:
df_for_teste.printSchema()

root
 |-- competÃªnciamov: long (nullable = true)
 |-- regiÃ£o: long (nullable = true)
 |-- uf: long (nullable = true)
 |-- municÃ­pio: long (nullable = true)
 |-- seÃ§Ã£o: string (nullable = true)
 |-- subclasse: long (nullable = true)
 |-- saldomovimentaÃ§Ã£o: long (nullable = true)
 |-- cbo2002ocupaÃ§Ã£o: long (nullable = true)
 |-- categoria: long (nullable = true)
 |-- graudeinstruÃ§Ã£o: long (nullable = true)
 |-- idade: long (nullable = true)
 |-- horascontratuais: string (nullable = true)
 |-- raÃ§acor: long (nullable = true)
 |-- sexo: long (nullable = true)
 |-- tipoempregador: long (nullable = true)
 |-- tipoestabelecimento: long (nullable = true)
 |-- tipomovimentaÃ§Ã£o: long (nullable = true)
 |-- tipodedeficiÃªncia: long (nullable = true)
 |-- indtrabintermitente: long (nullable = true)
 |-- indtrabparcial: long (nullable = true)
 |-- salÃ¡rio: string (nullable = true)
 |-- tamestabjan: long (nullable = true)
 |-- indicadoraprendiz: long (nullable = true)
 |-- origemdainf

In [23]:
# ========================================
# 3. Transformação
# ========================================

# ---- Ajuste dos nomes das colunas
rename_map = {
    "competÃªnciamov": "competenciamov",
    "competÃªnciadec": "competenciadec",
    "competÃªnciaexc": "competenciaexc",
    "regiÃ£o": "regiao",
    "municÃ­pio": "municipio",
    "seÃ§Ã£o": "secao",
    "saldomovimentaÃ§Ã£o": "saldomovimentacao",
    "cbo2002ocupaÃ§Ã£o": "cbo2002ocupacao",
    "graudeinstruÃ§Ã£o": "graudeinstrucao",
    "raÃ§acor": "racacor",
    "salÃ¡rio": "salario",
    "tipomovimentaÃ§Ã£o": "tipomovimentacao",
    "tipodedeficiÃªncia": "tipodeficiencia",
    "unidadesalÃ¡riocÃ³digo": "unidadesalariocodigo",
    "valorsalÃ¡riofixo": "valorsalariofixo",
    "indicadordeexclusÃ£o": "indicadordeexclusao"
}

# ---- Lê e adiciona flag de origem
def read_caged(path, origem):
    return (spark.read.parquet(path)
            .withColumn("origem_declaracao", lit(origem)))


for ano in anos:
    print(f"\n==== PROCESSANDO ANO {ano} ====")

    path_mov = f"gs://bucket_caged/caged/{ano}/MOV/{ano}MOV.parquet"
    path_for = f"gs://bucket_caged/caged/{ano}/FOR/{ano}FOR.parquet"
    path_exc = f"gs://bucket_caged/caged/{ano}/EXC/{ano}EXC.parquet"

    df_mov = read_caged(path_mov, "PRAZO")
    print(f"[INFO] Leitura Movimentação no prazo - Ano {ano}")

    df_for = read_caged(path_for, "FORA_PRAZO")
    print(f"[INFO] Leitura Movimentação fora do prazo - Ano {ano}")

    df_exc = read_caged(path_exc, "EXCLUSAO")
    print(f"[INFO] Leitura Exclusão - Ano {ano}")

    # MOV + FOR + EXC
    df = df_mov.unionByName(df_for, allowMissingColumns=True).unionByName(df_exc, allowMissingColumns=True)
    print(f"[INFO] União das origens OK!")

    # Renomeia colunas
    for old, new in rename_map.items():
        if old in df.columns:
            df = df.withColumnRenamed(old, new)
            print("[INFO] Colunas renomeadas!")

    # Criação de outras variáveis
    # -- Anos
    df = df.withColumn("ano_mov", substring("competenciamov", 1, 4))
    print("[INFO] Variavel ano_mov criada.")

    df = df.withColumn("ano_dec", substring("competenciadec", 1, 4))
    print("[INFO] Variavel ano_dec criada.")


    # -- Natureza do evento
    df = df.withColumn(
        "natureza_evento",
        when(col("saldomovimentacao") == 1, "admissao")
        .when(col("saldomovimentacao") == -1, "desligamento")
        .otherwise("desconhecido")
    )
    print("[INFO] Variavel natureza_evento criada.")

    # -- Impacto da exclusão no saldo
    df = df.withColumn(
        "impacto_saldo",
        when(col("origem_declaracao") == "EXCLUSAO", -col("saldomovimentacao"))
        .otherwise(col("saldomovimentacao"))
    )
    print("[INFO] Variavel impacto_saldo criada.")

    # -- Faixa etária
    df = df.withColumn(
        "faixa_etaria",
        when(col("idade") <= 18, "Ate 18")
        .when((col("idade") >= 19) & (col("idade") <= 24), "19-24")
        .when((col("idade") >= 25) & (col("idade") <= 34), "25-34")
        .when((col("idade") >= 35) & (col("idade") <= 44), "35-44")
        .when((col("idade") >= 45) & (col("idade") <= 59), "45-59")
        .when(col("idade") >= 60, "60+")
        .otherwise("Desconhecido")
    )
    print("[INFO] Variavel faixa_etaria criada.")

    # Append no bucket
    df.write.mode("append") \
        .partitionBy("ano_dec", "competenciadec", "origem_declaracao") \
        .parquet("gs://bucket_caged/caged_unificado/")

    print("[INFO] Append .parquet gs://bucket_caged/caged_unificado/ OK!")

print(".    Pipeline concluída!")



==== PROCESSANDO ANO 2020 ====
[INFO] Leitura Movimentação no prazo - Ano 2020
[INFO] Leitura Movimentação fora do prazo - Ano 2020
[INFO] Leitura Exclusão - Ano 2020
[INFO] União das origens OK!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Colunas renomeadas!
[INFO] Variavel ano_mov criada.
[INFO] Variavel ano_dec criada.
[INFO] Variavel natureza_evento criada.
[INFO] Variavel impacto_saldo criada.
[INFO] Variavel faixa_etaria criada.
[INFO] Append .parquet gs://bucket_caged/caged_unificado/ OK!

==== PROCESSANDO ANO 2021 ====
[INFO] Leitura Movimentação no prazo - Ano 2021
[INFO] Leitura Movimentação fora do prazo - Ano 20